In [1]:
import time
class Timer:
    def __init__(self, s, iters):
        self.s = s
        self.iters = iters
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        print("\"" + self.s + "\"", "took", (time.perf_counter() - self.start)/self.iters*1e9, "nanoseconds per db row")

In [2]:
import numpy as np
import pandas as pd

from starter import compute_fingerprint, load_database

with Timer("db loading", 1000000):
    fingerprints = load_database("database_fingerprints.npy")
    molecules = pd.read_csv("database.csv")
with open("query.txt") as q:
  query = q.readline()

"db loading" took 2373.546700022416 nanoseconds per db row


In [3]:
k = 5

# Warmup
Just python!

In [4]:
def coverage(query, ref):
  query_f = compute_fingerprint(query)
  ref_f = compute_fingerprint(ref)
  return sum(query_f & ref_f) / sum(query_f)

with Timer("python coverage function", 1000):
    scores = [coverage(query, molecules["smiles"][i]) for i in range(1000)]
    topk = np.argsort(scores)[-k:]
print(scores[:k])
print(topk, [scores[i] for i in topk])

"python coverage function" took 2317983.599990839 nanoseconds per db row
[0.2515592515592516, 0.22245322245322247, 0.2785862785862786, 0.22453222453222454, 0.30353430353430355]
[928  66  59 429 428] [0.4261954261954262, 0.4303534303534304, 0.4365904365904366, 0.46361746361746364, 0.498960498960499]


# Some sanity checks before we proceed
After we check that these scores are meaningful, we can proceed and just verify that our scores are exactly identical to this baseline.

In [5]:
print(coverage(molecules["smiles"][0], molecules["smiles"][0])) # should be 1
print(coverage(molecules["smiles"][0], molecules["smiles"][1])) # should be between 0 and 1
print(coverage("c1ccccc1", "C1CCCCC1")) # should be 0

1.0
0.746268656716418
0.0


# Speed things up with NumPy
We can get a massive speedup by using NumPy; everything is automatically vectorized.

In [6]:
with Timer("numpy direct bitwise_and", 100000):
    query_f = compute_fingerprint(query)
    scores = np.sum(query_f & fingerprints[:100000], axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy direct bitwise_and" took 6664.069000107702 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[94115 39023 94179 94185 38955] [0.60914761 0.59459459 0.6029106  0.6008316  0.5966736 ]


# Packing bits
Packing bits allows us to `&` 8 bits at a time.

Unpacking and summing is extremely inefficient though.

In [7]:
with Timer("numpy packed bits bitwise_and", 100000):
    query_f = compute_fingerprint(query)
    scores = np.sum(np.unpackbits(np.packbits(fingerprints[:100000], axis=1) & np.packbits(query_f)).reshape(-1, 2048), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy packed bits bitwise_and" took 2031.6820000880396 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[94115 39023 94179 94185 38955] [0.60914761 0.59459459 0.6029106  0.6008316  0.5966736 ]


# Cython
We need C++ to get even faster; I want to use the native popcount instruction because [NumPy doesn't have a function to do that yet](https://github.com/numpy/numpy/issues/16325).

In [8]:
# Note: -O3 -march=native isn't just for fun, you really do need it to beat NumPy.
! which cython || conda install -y cython
! test ! -d $$CONDA_PREFIX/envs/myenv/lib/python3.6/site-packages/blosc || conda install -y python-blosc
! cython popcount.pyx
! sed -i '/0xbad0bad0/d' popcount.c
CXX="g++"
! $CXX --version
#! g++ -Wall -O3 -Ofast -g -flto -lm -L$$CONDA_PREFIX/lib -lblosc -shared -pthread -fPIC -funroll-loops -fno-strict-aliasing -march=native -mno-avx256-split-unaligned-load -fopt-info-vec-optimized -I$$CONDA_PREFIX/include/python3.6m -I$$CONDA_PREFIX/include -I$$CONDA_PREFIX/lib/python3.6/site-packages/numpy/core/include/ -o popcount.so popcount.c
! $CXX -Wall -Ofast -g -flto -fPIC -funroll-loops -fopenmp -fwrapv -fno-strict-aliasing -march=native -I$$CONDA_PREFIX/include/python3.7m -I$$CONDA_PREFIX/include -I$$CONDA_PREFIX/lib/python3.7/site-packages/numpy/core/include -c -o popcount.o popcount.c
! $CXX -Wall -Ofast -g -flto -shared -pthread -fPIC -funroll-loops -fopenmp -fwrapv -fno-strict-aliasing -march=native -o popcount.so popcount.o -L$$CONDA_PREFIX/lib -Wl,-rpath=$$CONDA_PREFIX/lib -lblosc


/home/clive/anaconda3/envs/my-rdkit-env/bin/cython
/home/clive/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/clive/code/reverie-challenge/popcount.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
g++ (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

In file included from /home/clive/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822,
                 from /home/clive/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/clive/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/numpy/core/include/numpy

In [9]:
from popcount import inplace_popcount_32
with Timer("numpy packed bits popcounted", 1000000):
    query_f = compute_fingerprint(query)
    isct = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1) & np.packbits(query_f), dtype=np.uint32)
    inplace_popcount_32(isct)
    scores = np.sum(isct.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy packed bits popcounted" took 435.0261999934446 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


Now that we're processing more than 5x faster than it takes to read the database off the SSD, we can declare victory, but also I'm curious how far we can go.

# Fusing operations
This is sort of a diminishing returns scenario; I needed `-O3 -march=native` to run the bitwise and faster than NumPy.

In [10]:
from popcount import fused_popcount_bitwise_and
with Timer("numpy fused bitwise_and + popcount", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
    fused_popcount_bitwise_and(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy fused bitwise_and + popcount" took 394.0944000205491 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Fusing the packing too
The packbits function is now the slowest part, at about 80% of the runtime. What if we don't do that at all?

It ends up being slower than NumPy because NumPy is [using proper SIMD instructions for this](https://github.com/numpy/numpy/blob/97d2db483fc0ffd46f38d0e1c39d5fc001e33197/numpy/core/src/multiarray/compiled_base.c#L1543). This is okay, we now have the tools to do this properly.

In [11]:
from popcount import fused_popcount_bitwise_and_notpacked_count
with Timer("fused notpacked popcount", 1000000):
    query_f = compute_fingerprint(query).astype(np.uint8)
    fused_popcount_bitwise_and_notpacked_count(query_f, fingerprints[:1000000])
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"fused notpacked popcount" took 1220.9746999724302 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Endgame: AVX2
AVX2 is supported by most modern processors, including the Intel processor that the specified MBP 2019 has.

We get an absurdly high throughput: we're processing about 5 boolean database entries per nanosecond.

In [12]:
from popcount import fused_popcount_avx2
with Timer("avx2", 1000000):
    query_f = compute_fingerprint(query).astype(np.uint8)
    scores = fused_popcount_avx2(query_f, fingerprints[:1000000]) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"avx2" took 123.41299999388865 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Comments and things to improve

Algorithmic complexity is O(database size), since it must iterate over all the booleans in every row. I'm fairly certain that this cannot be asymptotically improved - for example in the worst case every molecule in the database contains all substructures (all 2048 bits are true).

Memory usage is high; we're reading the entire database into memory and keeping it there. There isn't really any way around keeping O(database size) in memory because SSD bandwidth is far too low to support streaming from disk, but an 8x memory usage/bandwidth bottleneck improvement can probably be found by bitpacking the database beforehand.

# Bonus: Overtime stuff

This was done for fun significantly over the 6hr time limit; please don't count this if you're evaluating quantity of code produced.

## Pre-packing

I did no memory-related optimizations mostly because I was more interested in playing with AVX, and because up until AVX the whole thing was compute-bottlenecked rather than memory-bottlenecked. However it's trivial to do so; just don't count the bitpacking step from the `Fusing operations` section, and you get (only!) a slight speedup over the AVX implementation as seen below.

In [13]:
from popcount import fused_popcount_bitwise_and
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
with Timer("pre-packed rows", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
    fused_popcount_bitwise_and(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"pre-packing process" took 238.83260000729933 nanoseconds per db row
"pre-packed rows" took 123.92220000037923 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


## AVX2 popcount attempt 1

Not working correctly, but AVX2 emulated popcnt [can be 30% faster](https://stackoverflow.com/a/50082218) than the dedicated instruction.

Edit: I think it won't be faster; according to [this paper](https://arxiv.org/pdf/1611.07612.pdf) and other sources, popcnt on 64-bit ints is the fastest way to do it at our data size of 2048 bits.

In [14]:
from popcount import fused_avx2_emulated_popcount
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint64)
with Timer("pre-packed rows", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint64)
    fused_avx2_emulated_popcount(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"pre-packing process" took 274.39370000502095 nanoseconds per db row
"pre-packed rows" took 57.00010000145994 nanoseconds per db row
[4.03833063e+15 2.62815452e+16 3.77945367e+15 3.34960946e+16
 8.88579474e+15]
[439306  28215  93292 489702 106223] [3.83505627e+16 3.83506231e+16 3.83505872e+16 3.83507498e+16
 3.83504301e+16]


## Popcount64 and some tweaks to get good assembly

Let's use popcount64 directly on 64-bit ints, instead of 32-bit ints, and also do counting in the kernel itself. (Also added loop unrolling flag, for the slight benefit of all these examples.)

Maybe let's be less guessy. Here's a relevant assembly sample from a single iteration of the unrolled loop.
```
   27c1a:       44 09 c2                or     %r8d,%edx
   27c1d:       41 89 07                mov    %eax,(%r15)
   27c20:       48 63 ca                movslq %edx,%rcx
   27c23:       49 0f af cc             imul   %r12,%rcx
   27c27:       49 8b 14 0b             mov    (%r11,%rcx,1),%rdx
   27c2b:       48 23 17                and    (%rdi),%rdx
   27c2e:       31 c9                   xor    %ecx,%ecx
   27c30:       4c 01 cf                add    %r9,%rdi
   27c33:       f3 48 0f b8 ca          popcnt %rdx,%rcx
   27c38:       8d 56 02                lea    0x2(%rsi),%edx
   27c3b:       01 c8                   add    %ecx,%eax
```
There might be a tiny bit of gain to be had by avoiding some of these instructions. I don't fully understand what they do, though.

Making `count` as a separate int variable (instead of accumulating directly into `counts[i>>5]`) results in much less consistent loop unrolling - there doesn't actually seem to be a particular pattern to the iterations; they have a bunch of random instructions squeezed in between that don't necessarily repeat. Finally, making `fingerprints_packed_curr` as a temporary variable instead of calculating `i|j` every time makes the loop much tighter:
```
   27d17:       4c 8b 79 d0             mov    -0x30(%rcx),%r15    ; Get a quadword of fingerprints_packed_curr (fixed offset)
   27d1b:       4c 23 3a                and    (%rdx),%r15         ; AND it with a query quadword
   27d1e:       48 01 f2                add    %rsi,%rdx           ; Advance the query pointer
   27d21:       f3 4d 0f b8 ff          popcnt %r15,%r15           ; Popcount it
   27d26:       41 01 c7                add    %eax,%r15d          ; Add this to the count
```
Those two modifications combined give an additional 25-30% better performance. I don't think there's anything else that can be done to speed this up, unless AVX2 has some real magic to offer (but even then, the frequency hit from AVX/AVX2 probably means that it won't be able to overcome this).

The fastest I've seen for just the `fused_popcount64_bitwise_and` part is 30ns:
- 2048bits/30ns = 8.5 GBps, out of a memory bandwidth of ~10-20GBps.
- 32 popcounts / 30ns = approx one 64-bit popcount per 0.9 nanoseconds. This almost double the expected CPI of 5 cycles @ 3GHz = 1.6 ns, assuming the 5 listed instructions above take 1 cycle each, but apparently this is very superscalar, or my math is wrong (is frequency == cycles per second?)

In [15]:
from popcount import fused_popcount64_bitwise_and
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='little'), dtype=np.uint64)
with Timer("pre-packed rows", 10000000):
    for i in range(10):
        query_f = compute_fingerprint(query)
        query_packed = np.frombuffer(np.packbits(query_f, bitorder='little'), dtype=np.uint64)
        counts = fused_popcount64_bitwise_and(query_packed, fingerprints_packed)
        total = fused_popcount64_bitwise_and(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
        topk = np.argpartition(-counts, k)[:k]
print(counts[:k]/total)
print(topk, counts[topk]/total)

"pre-packing process" took 212.23840001039207 nanoseconds per db row
"pre-packed rows" took 27.69667999818921 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


## AVX2 attempt 2: success and maximum throughput

It might be possible to use AVX2 instructions to MOV or AND or ADD more efficiently; 10ns is the absolute limit because the popcount instruction has a CPI of 1. I'm highly doubtful it'll make things faster given that the width is only 256 bits anyway though, and there has to be some penalty moving between AVX registers and the regular registers where popcnt lives, though maybe that can be pipelined away.
- LOADU twice for 256 bits each
- AND 256 bits together
- _mm256_extract_epi64 and popcount each of the 4 registers, adding

With some guidance from [this](https://github.com/WojciechMula/sse-popcount/blob/master/popcnt-avx2-lookup.cpp) we can get a 20% speed boost.

It turns out that even though benchmarks show unrolled loop of int64 popcnt is the fastest popcount implementation for 256 bytes, for our usecase we have additional math alongside it (the AND) so putting everything into AVX is actually beneficial.

Our inner loop looks something like ("something like" because loop unrolling has made things confusing) this:
```
   28b23:       c4 43 25 38 6d 10 01    vinserti128 $0x1,0x10(%r13),%ymm11,%ymm13
   28b2a:       c5 d5 71 d4 04          vpsrlw $0x4,%ymm4,%ymm5
   28b2f:       c4 63 35 38 54 07 50    vinserti128 $0x1,0x50(%rdi,%rax,1),%ymm9,%ymm10
   28b36:       01 
   28b37:       c4 c1 7d fc c7          vpaddb %ymm15,%ymm0,%ymm0
   28b3c:       c5 6d db c5             vpand  %ymm5,%ymm2,%ymm8
   28b40:       c5 dd db f2             vpand  %ymm2,%ymm4,%ymm6
   28b44:       c4 c1 15 db ca          vpand  %ymm10,%ymm13,%ymm1
   28b49:       c4 42 1d 00 f0          vpshufb %ymm8,%ymm12,%ymm14
   28b4e:       c4 41 7a 6f 14 24       vmovdqu (%r12),%xmm10
   28b54:       c4 e2 1d 00 fe          vpshufb %ymm6,%ymm12,%ymm7
   28b59:       c5 7a 6f 44 07 60       vmovdqu 0x60(%rdi,%rax,1),%xmm8
```
Even though it's longer, it's also handling 4x as much data per iteration.

Turning on `-march=native -mno-avx256-split-unaligned-load` gets us to something like
```
   289ac:       c4 41 45 db cd          vpand  %ymm13,%ymm7,%ymm9
   289b1:       c5 7d fc fe             vpaddb %ymm6,%ymm0,%ymm15
   289b5:       c5 ed db bc 07 a0 00    vpand  0xa0(%rdi,%rax,1),%ymm2,%ymm7
   289bc:       00 00 
   289be:       c4 c1 05 fc f6          vpaddb %ymm14,%ymm15,%ymm6
   289c3:       c4 e2 1d 00 e1          vpshufb %ymm1,%ymm12,%ymm4
   289c8:       c4 c1 15 db c8          vpand  %ymm8,%ymm13,%ymm1
   289cd:       c4 c2 1d 00 d9          vpshufb %ymm9,%ymm12,%ymm3
   289d2:       c5 bd 71 d7 04          vpsrlw $0x4,%ymm7,%ymm8
```
Where we only see instructions that we've specified explicitly in the code. In fact, we're missing a bunch of `vmovdqu`'s - the compiler is doing something clever here that I don't understand.

- 2048bits/22ns = 11.6 GBps, out of a memory bandwidth of ~10-20GBps.
- ~80 AVX instructions / 22ns = approx 3.6GHz, which is significantly higher than the clock frequency. This is probably because some instructions can have a throughput of more than 1 per cycle.

This may well be the end of the line for optimizing this, as this does seem to be optimal (all sources point to Mula as the source of these algorithms, and this is the best his papers have to offer)

In [16]:
from popcount import fused_popcount64_bitwise_and_avx
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='little'), dtype=np.uint64)
with Timer("pre-packed rows 1", 10000000):
    for i in range(10):
        query_f = compute_fingerprint(query)
with Timer("pre-packed rows 2", 10000000):
    for i in range(10):
        query_packed = np.frombuffer(np.packbits(query_f, bitorder='little'), dtype=np.uint64)
with Timer("pre-packed rows 3", 10000000):
    for i in range(10):
        counts = fused_popcount64_bitwise_and_avx(query_packed, fingerprints_packed)
with Timer("pre-packed rows 4", 10000000):
    for i in range(10):
        total = fused_popcount64_bitwise_and_avx(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
with Timer("pre-packed rows 5", 10000000):
    for i in range(10):
        topk = np.argpartition(-counts, k)[:k]
print(counts[:k]/total)
print(topk, counts[topk]/total)

"pre-packing process" took 207.71160000003874 nanoseconds per db row
"pre-packed rows 1" took 0.9360399999422953 nanoseconds per db row
"pre-packed rows 2" took 0.023940001847222447 nanoseconds per db row
"pre-packed rows 3" took 18.461230001412332 nanoseconds per db row
"pre-packed rows 4" took 0.004290000651963055 nanoseconds per db row
"pre-packed rows 5" took 4.68531999795232 nanoseconds per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


## Finale: Optimizing other parts of the system

We're actually getting to the point where the final np.argpartition on 1 million elements is taking about 25% of the total time. Using np.argsort is 10x slower. We did actually need this initially seemingly unnecessary optimization!

There are [ways](https://github.com/WojciechMula/simd-sort) to make this happen faster, but since we're going to be generating the entire count array anyway, why not just construct it on the fly? Specifically, let's use a **min-heap of size k** to track the k largest elements we've seen.

This absorbs the topk algorithm without any measurable performance penalty (less than 10% with k=1000). A bit of tweaking suffices to double the speed of the compute_fingerprint, and now we are truly at the end of the line.

In [17]:
from popcount import fused_popcount64_bitwise_and_avx_topk, fused_popcount64_bitwise_and_avx
from rdkit import Chem

with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='big'), dtype=np.uint64)
with Timer("pre-packed rows 0", 100000000):
    for i in range(100):
        mol = Chem.MolFromSmiles(query)
with Timer("pre-packed rows 1", 100000000):
    for i in range(100):
        s = Chem.RDKFingerprint(mol, fpSize=2048, maxPath=5).ToBitString()
with Timer("pre-packed rows 2", 100000000):
    for i in range(100):
        query_packed = np.copy(np.frombuffer(int(s, 2).to_bytes(len(s) // 8, byteorder='big'), dtype=np.uint64))
for i in range(100):
    with Timer("pre-packed rows 3", 1000000):
        counts, topk = fused_popcount64_bitwise_and_avx_topk(query_packed, fingerprints_packed, k)
with Timer("pre-packed rows 4", 100000000):
    for i in range(100):
        total = fused_popcount64_bitwise_and_avx(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
print(topk, counts/total)

"pre-packing process" took 248.92470001941547 nanoseconds per db row
"pre-packed rows 0" took 0.1470180001342669 nanoseconds per db row
"pre-packed rows 1" took 0.32450899976538494 nanoseconds per db row
"pre-packed rows 2" took 0.015281999949365854 nanoseconds per db row
"pre-packed rows 3" took 17.00739999068901 nanoseconds per db row
"pre-packed rows 3" took 17.381500016199425 nanoseconds per db row
"pre-packed rows 3" took 16.747099987696856 nanoseconds per db row
"pre-packed rows 3" took 19.762799987802282 nanoseconds per db row
"pre-packed rows 3" took 18.822899990482256 nanoseconds per db row
"pre-packed rows 3" took 17.291199997998774 nanoseconds per db row
"pre-packed rows 3" took 17.290799994952977 nanoseconds per db row
"pre-packed rows 3" took 16.846799990162253 nanoseconds per db row
"pre-packed rows 3" took 16.25640000565909 nanoseconds per db row
"pre-packed rows 3" took 16.6742000146769 nanoseconds per db row
"pre-packed rows 3" took 16.57030000933446 nanoseconds per db

## OpenMP Threading

After some OpenMP we can smash the 20ns barrier.

"personal best" = "5.8700999999814485 nanoseconds per db row" for `fused_popcount64_bitwise_and_avx_topk_omp`

3600MHz RAM, 2048 bits / 5.869199999978036 ns = 43.6175288 GBps, after moving to my Ryzen 3700x machine which should have a max of 28.8GBps per stick, times two sticks.

Going from 2133MHz to 3600MHz speeds up by 50%, and overclocking CPU doesn't visibly help, so it's legitimately memory bandwidth bound, and even after the frequency increase to 3600MHz it's still probably memory bound. That's pretty wild.

In [18]:
from popcount import fused_popcount64_bitwise_and_avx_topk_omp, fused_popcount64_bitwise_and_avx
from rdkit import Chem

import os
# This only works before the first run of openmp in a process
os.environ["OMP_PLACES"] = "cores"
os.environ["OMP_NUM_THREADS"] = "8"

with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='big'), dtype=np.uint64)
with Timer("pre-packed rows 0", 100000000):
    for i in range(100):
        mol = Chem.MolFromSmiles(query)
with Timer("pre-packed rows 1", 100000000):
    for i in range(100):
        s = Chem.RDKFingerprint(mol, fpSize=2048, maxPath=5).ToBitString()
with Timer("pre-packed rows 2", 100000000):
    for i in range(100):
        query_packed = np.copy(np.frombuffer(int(s, 2).to_bytes(len(s) // 8, byteorder='big'), dtype=np.uint64))
for i in range(50):
    with Timer("pre-packed rows 3", 10000000):
        for i in range(10):
            counts, topk = fused_popcount64_bitwise_and_avx_topk_omp(query_packed, fingerprints_packed, k)
with Timer("pre-packed rows 4", 100000000):
    for i in range(100):
        total = fused_popcount64_bitwise_and_avx(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
print(topk, counts/total)

"pre-packing process" took 247.27249998250045 nanoseconds per db row
"pre-packed rows 0" took 0.15130400017369539 nanoseconds per db row
"pre-packed rows 1" took 0.3249610000057146 nanoseconds per db row
"pre-packed rows 2" took 0.014942999987397343 nanoseconds per db row
"pre-packed rows 3" took 12.409160000970587 nanoseconds per db row
"pre-packed rows 3" took 6.690320000052452 nanoseconds per db row
"pre-packed rows 3" took 6.4921800018055364 nanoseconds per db row
"pre-packed rows 3" took 6.844000000273809 nanoseconds per db row
"pre-packed rows 3" took 7.584119998500682 nanoseconds per db row
"pre-packed rows 3" took 7.036680000601336 nanoseconds per db row
"pre-packed rows 3" took 7.063479998032562 nanoseconds per db row
"pre-packed rows 3" took 6.535829999484122 nanoseconds per db row
"pre-packed rows 3" took 6.469980001566 nanoseconds per db row
"pre-packed rows 3" took 6.811080002808012 nanoseconds per db row
"pre-packed rows 3" took 6.6672899993136525 nanoseconds per db row
"

## Failed: blosc
Since we might have a memory bottleneck, does blosc help?

Nope, that was a total failure. Maybe next step is to try out custom encodings like RLE or even sparse bits.

In [19]:
import blosc
fingerprints_blosc_compressed = np.frombuffer(blosc.compress(fingerprints_packed, typesize=8, shuffle=blosc.NOSHUFFLE, cname='blosclz'), dtype=np.uint8)

print("Compression:", len(fingerprints_blosc_compressed)/8/len(fingerprints_packed))
# %timeit blosc.decompress(fingerprints_blosc_compressed)
# %timeit np.copy(fingerprints_packed)

Compression: 0.70317598828125


In [20]:
from popcount import fused_popcount64_bitwise_and_avx_topk_omp_blosc, fused_popcount64_bitwise_and_avx
from rdkit import Chem
import blosc

with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints, axis=1, bitorder='big'), dtype=np.uint64)
    fingerprints_blosc_compressed = np.frombuffer(blosc.compress(fingerprints_packed, typesize=8, shuffle=blosc.NOSHUFFLE, cname='blosclz'), dtype=np.uint8)
    n_fingerprints = len(fingerprints)
with Timer("pre-packed rows 0", 1000000):
    for i in range(1):
        mol = Chem.MolFromSmiles(query)
with Timer("pre-packed rows 1", 1000000):
    for i in range(1):
        s = Chem.RDKFingerprint(mol, fpSize=2048, maxPath=5).ToBitString()
with Timer("pre-packed rows 2", 1000000):
    for i in range(1):
        query_packed = np.copy(np.frombuffer(int(s, 2).to_bytes(len(s) // 8, byteorder='big'), dtype=np.uint64))
with Timer("pre-packed rows 3", 1000000):
    for i in range(1):
        counts, topk = fused_popcount64_bitwise_and_avx_topk_omp_blosc(query_packed, fingerprints_blosc_compressed, k, n_fingerprints)
with Timer("pre-packed rows 4", 1000000):
    for i in range(1):
        total = fused_popcount64_bitwise_and_avx(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
print(topk, counts/total)

"pre-packing process" took 605.843100027414 nanoseconds per db row
"pre-packed rows 0" took 0.2631999959703535 nanoseconds per db row
"pre-packed rows 1" took 0.3476999991107732 nanoseconds per db row
"pre-packed rows 2" took 0.05319999763742089 nanoseconds per db row
"pre-packed rows 3" took 61.555499996757135 nanoseconds per db row
"pre-packed rows 4" took 0.020899984519928694 nanoseconds per db row
[ 999437 1003532  999438 1003533 1003534] [1. 1. 1. 1. 1.]


## The real endgame: CUDA

In [21]:
! pip install pycuda

In [22]:
import pycuda
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda.gpuarray import GPUArray, to_gpu

In [23]:
with Timer("pre-packing process", 1000000):
    fingerprints_packed_gpu = to_gpu(np.frombuffer(np.packbits(fingerprints, axis=1, bitorder='big'), dtype=np.uint64))
    print(fingerprints_packed_gpu.shape)
    sums_gpu = GPUArray((100,), dtype=np.int32)
    n_fingerprints = len(fingerprints)

(32000000,)
"pre-packing process" took 318.59070001519285 nanoseconds per db row


In [24]:
mod = SourceModule("""
  __global__ void and_popcount(unsigned long long *a, unsigned long long *b, int *sums)
  {
    int START = (blockIdx.x * blockDim.x + threadIdx.x) * 1000;
    int END = min((blockIdx.x * blockDim.x + threadIdx.x + 1) * 1000, 1000000);
    int sum = 0;
    for (int i = START; i < START+1; i++) {
      unsigned long long *a_tmp = &a[i<<5];
      for (int j = 0; j < 32; j++) {
        sum += __popcll(a_tmp[j] & b[j]);
      }
    }
    sums[blockDim.x * blockIdx.x + threadIdx.x] = sum;
  }
  """, options=['--use_fast_math', '-O3', '-Xptxas', '-O3,-v'])
and_popcount = mod.get_function("and_popcount")

In [25]:
from rdkit import Chem
mol = Chem.MolFromSmiles(query)
s = Chem.RDKFingerprint(mol, fpSize=2048, maxPath=5).ToBitString()
query_packed_gpu = to_gpu(np.frombuffer(int(s, 2).to_bytes(len(s) // 8, byteorder='big'), dtype=np.uint64))
print(query_packed_gpu.shape)

and_popcount(fingerprints_packed_gpu, query_packed_gpu, sums_gpu, grid=(32,1,1), block=(32,1,1))
pycuda.gpuarray.sum(sums_gpu)

(32,)


LogicError: cuModuleLoadDataEx failed: an illegal memory access was encountered - 

In [ ]:
from rdkit import Chem

with Timer("pre-packed rows 0", 1000000):
    for i in range(1):
        mol = Chem.MolFromSmiles(query)
with Timer("pre-packed rows 1", 1000000):
    for i in range(1):
        s = Chem.RDKFingerprint(mol, fpSize=2048, maxPath=5).ToBitString()
with Timer("pre-packed rows 2", 1000000):
    for i in range(1):
        query_packed = np.copy(np.frombuffer(int(s, 2).to_bytes(len(s) // 8, byteorder='big'), dtype=np.uint64))
with Timer("pre-packed rows 3", 1000000):
    for i in range(1):
        counts, topk = fused_popcount64_bitwise_and_avx_topk_omp_blosc(query_packed, fingerprints_blosc_compressed, k, n_fingerprints)
with Timer("pre-packed rows 4", 1000000):
    for i in range(1):
        total = fused_popcount64_bitwise_and_avx(query_packed, query_packed) # About twice as fast as np.sum(query_packed)
print(topk, counts/total)

## Future stuff
AVX-512 would be fun but I don't have ice lake hardware sadly. This should be something like 3x faster, less any AVX-induced processor frequency hit.
- `_mm512_loadu_epi64`
- `_mm512_and_epi64`
- `_mm512_popcnt_epi64`
- `_mm512_reduce_add_epi64`

Some additional cleverness that I do not have time to explore, but which is unlikely to yield better results:
- Database rows are almost always sparse. There might be more efficient popcount algorithms in this case.
- OpenCL was originally planned but no time, and also I think my laptop iGPU doesn't support it anyway.

# Python Threading
Because why not? The AVX2 code is mostly in C-land away from the GIL so we might be able to get close to linear speedup in the number of cores (regular cores; hyperthreading probably does not help for this because AVX2 resources are shared).

This didn't help at all with the AVX2 code because there are other bottlenecks e.g. system memory bandwidth. 2GB dataset in 0.2s is 10GBps, which is around half the theoretical memory bandwidth my machine could have, but there's probably some other caveats there.

This does help a lot (+50% ish) with regular popcount64 because of the 8x lower memory bandwidth requirement when bits are packed. 2GB/8 = 256MB dataset in 0.08s is only 3GBps, so it's probably still compute limited and could benefit from more threading. Unfortunately I only have 2 actual cores (4 hyperthreaded) so there can be no more than a 2x speedup.

However modifying popcount64 to fuse the counting step as well results in having no significant threading benefit. Occasionally it will run faster than single-threaded, but most of the time it's running into some issue or another - maybe memory bandwidth, maybe scheduler issues because it's so fast.

This also points toward there being very little point in using OpenCL: unless you have a discrete GPU with dedicated high bandwidth video memory, which isn't the case on MBP 2019, it's not going to get you much further, and more likely will slow things down.

In [ ]:
import threading
from popcount import fused_popcount64_bitwise_and
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='little'), dtype=np.uint64)

core_ids = set()
with open('/proc/cpuinfo') as f:
    for line in f:
        if line.count('core id'):
            core_ids.add(line)

nthreads = len(core_ids)
assert(len(fingerprints) % nthreads == 0) # Note: below code calculating start and end assumes that nthreads divides len(fingerprints)
print(f"Using {nthreads} threads")

topk_per_thread = [None] * nthreads
topk_per_thread_scores = [None] * nthreads

def thread_func(query_packed, threadid):
    start = threadid*len(fingerprints_packed)//nthreads
    end = (threadid+1)*len(fingerprints_packed)//nthreads
    
    counts = fused_popcount64_bitwise_and(query_packed, fingerprints_packed[start:end])
    total = fused_popcount64_bitwise_and(query_packed, query_packed)

    topk_per_thread[threadid] = np.argpartition(-counts, k)[:k]

    topk_per_thread_scores[threadid] = counts[topk_per_thread[threadid]] / total
    topk_per_thread[threadid] += start // 32

with Timer("threaded popcount64", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f, bitorder='little'), dtype=np.uint64)
    threads = [None] * nthreads
    for i in range(1, nthreads):
        threads[i] = threading.Thread(target=thread_func, args=(query_packed, i))
        threads[i].start()
    thread_func(query_packed, 0)
    for i in range(1, nthreads):
        threads[i].join()
    topk_arg = np.argsort(-np.concatenate(topk_per_thread_scores))[:k]
    scores = np.concatenate(topk_per_thread_scores)[topk_arg]
    topk = np.concatenate(topk_per_thread)[topk_arg]
print(topk, scores)

# Bonus: A failed attempt at using Numba
Cython has some ... interoperability issues ... with Numba, so using the popcount kernel isn't possible.

Using purely Numba (without the popcount kernel) is way slower.

In [ ]:
try:
    from numba import jit
except ImportError:
    ! conda install -y numba
    from numba import jit

In [ ]:
import numba
# from numba.extending import get_cython_function_address
# import numpy.ctypeslib as npct
# import ctypes

# array_1d_uint32 = npct.ndpointer(dtype=np.uint32, ndim=1, flags='CONTIGUOUS')
# addr = get_cython_function_address("popcount", "_fused_popcount_bitwise_and")
# functype = ctypes.CFUNCTYPE(None, array_1d_uint32, array_1d_uint32)
# fused_popcount_bitwise_and = functype(addr)

@numba.jit(nopython=True)
def numba_func(query_f, fingerprints):
    return np.sum(query_f & fingerprints[:100000], axis=1) / np.sum(query_f)

# @numba.jit(nopython=True)
# def numba_func(query_f, fingerprints):
#     query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
#     fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
#     fused_popcount_bitwise_and(query_packed, fingerprints_packed)
#     return np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)

with Timer("numba", 100000):
    query_f = compute_fingerprint(query)
    scores = numba_func(query_f, fingerprints)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

# Misc

In [ ]:
import importlib
import popcount
popcount = importlib.reload(popcount)